## Desambiguación basada en definiciones
Esta solución nace de la suposición de que en la definición de una palabra suelen aparecer palabras relacionadas, y que en su jerarquía de hiperónimos hay palabras que aparecen en las definiciones de palabras relacionadas. A continuación un ejemplo.

In [2]:
from nltk.corpus import wordnet as wn
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Si se observa la palabra `gato` (tomando la acepción de gato doméstico), en su definición aparecen palabras como doméstico, felino o mamífero, y en su ruta de hiperónimos aparecen palabras como animal, vertebrado, carnívoro, etc.

In [19]:
palabra = "cat"
acepciones = wn.synsets(palabra)
print(acepciones[0].definition())
print(acepciones[0].hypernym_paths()[0])

feline mammal usually having thick soft fur and no ability to roar: domestic cats; wildcats
[Synset('entity.n.01'), Synset('physical_entity.n.01'), Synset('object.n.01'), Synset('whole.n.02'), Synset('living_thing.n.01'), Synset('organism.n.01'), Synset('animal.n.01'), Synset('chordate.n.01'), Synset('vertebrate.n.01'), Synset('mammal.n.01'), Synset('placental.n.01'), Synset('carnivore.n.01'), Synset('feline.n.01'), Synset('cat.n.01')]


Tomando otro sustantivo relacionado como puede ser `mascota` y observando sus definiciones se puede ver que la definición que más palabras relacionadas tiene con la acepción de `gato` y la ruta de hiperónimos tomada es la primera, donde aparece "domesticated" y "animal", por lo que sería seleccionada por el algoritmo

In [20]:
palabra = "pet"
acepciones = wn.synsets(palabra, pos=wn.NOUN)
for i in acepciones:
    print(i.name(), " ",i.definition())

pet.n.01   a domesticated animal kept for companionship or amusement
darling.n.01   a special loved one
pet.n.03   a fit of petulance or sulkiness (especially at what is felt to be a slight)
positron_emission_tomography.n.01   using a computerized radiographic technique to examine the metabolic activity in various tissues (especially in the brain)


### Explicación del algoritmo
El algoritmo recibe 2 palabras, y su objetivo es obtener las definiciones que estén más relacionadas. Para esto concatena las definiciones de cada palabra con su ruta de hiperónimos para tener la mayor cantidad de palabras relacionadas a al significado.

A continuación se realiza una vectorización y el cálculo de distancia coseno sobre la matriz resultante. De esta forma el resultado es una matriz donde hay tantas filas como acepciones tenga la primera palabra, y tantas columnas como acepciones tenga la segunda palabra. Los valores de esta matriz son la distancia coseno entre las acepciones.

De esta forma simplemente recorriendo la matriz se puede obtener las dos acepciones con la distancia coseno más baja, por lo que serán las más similares.

Este algoritmo fue descartado ya que en las pruebas la desambiguación basada en la métrica JNC dio mejores resultados.

In [1]:
def disambiguate_nouns(word1, word2):
    
    synsets_word1 = wn.synsets(word1, pos=wn.NOUN)
    synsets_word2 = wn.synsets(word2, pos=wn.NOUN)
    
    vectorizer = CountVectorizer()
    synset_definitions = [synset.definition() + ' ' + ' '.join([word.name() for word in synset.hypernym_paths()[0]])  for synset in synsets_word1]
    synset_definitions = synset_definitions + [synset.definition() + ' ' + ' '.join([word.name() for word in synset.hypernym_paths()[0]])  for synset in synsets_word2]
    vector_space = vectorizer.fit_transform(synset_definitions)
    similarity_matrix = cosine_similarity(vector_space)
    
    max_value = -1
    row_index = -1
    col_index = -1
    for row in range(len(synsets_word1)):
        for col in range(len(synsets_word1), len(synset_definitions)):
            if col != row:
                if similarity_matrix[row,col] > max_value:
                    max_value = similarity_matrix[row,col]
                    row_index = row
                    col_index = col

    
    best_synset_word1 = synsets_word1[row_index]
    best_synset_word2 = synsets_word2[col_index-len(synsets_word1)]

    return best_synset_word1, best_synset_word2

# Ejemplo de uso
word1 = "cat"
word2 = "dog"
best_synset_word1, best_synset_word2 = disambiguate_nouns(word1, word2)
print(best_synset_word1.definition())
print(best_synset_word2.definition())


any of several large cats typically able to roar and living in the wild
a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds
